# Notebook 01: GenAI Extraction and Document Chunking

This notebook starts the first stage of the end to end GenAI insurance document pipeline. The goal is to take a collection of long, unstructured documents and convert them into structured JSON outputs that can be analysed in later notebooks.

The raw dataset includes three document types stored in the data/raw folder:  
• Insurance policy wording style documents  
• ESG style sustainability reports  
• Incident and loss investigation summaries  

Each document is synthetic, created through paraphrasing or original generation so that it mimics the style, tone, and complexity of real insurance documents without raising copyright issues. The documents are intentionally messy, unstructured, and varied in length to reflect the challenges seen in real underwriting and claims workflows.

Insurance related documents are often long and difficult to process. Important details such as entities, regions, risk factors, exclusions, operational failures, and ESG concerns are spread across multiple paragraphs. Manual extraction is slow and inconsistent. This notebook focuses on building a reproducible GenAI workflow that can extract structured fields in a reliable way.

In this notebook we will:
1. Load the raw documents from data/raw  
2. Split each document into smaller chunks for safer extraction  
3. Define a consistent JSON schema for the key fields we want to extract  
4. Build a clear prompt template that enforces structure and avoids hallucination  
5. Run the extraction process using helper functions in src  
6. Apply a simple MapReduce style pattern by merging chunk outputs  
7. Validate and save the final JSON outputs into data/processed  

The goal is to produce clean, validated, and consistent structured data. These outputs will then be used in Notebook 02 for EDA and in Notebook 03 for feature engineering and classification. This mirrors practical workflows in insurtech companies, where large volumes of unstructured documents must be prepared and structured before modelling or decision support.


## Step 1: Set up imports and locate the raw documents

Before we can do any chunking or GenAI extraction, we need a reliable way to point the notebook at the `data/raw` folder and see exactly which files we are working with.

In this step we will:

- Import a small set of core Python utilities  
- Define a `PROJECT_ROOT` and `DATA_RAW_DIR` using `pathlib.Path` so paths are robust  
- List all text files in the three raw subfolders: `policies`, `esg`, and `incidents`  
- Store these file paths in a simple dictionary for later steps




In [1]:
from pathlib import Path

# 1) Define project root and raw data directory
PROJECT_ROOT = Path("..").resolve()
DATA_RAW_DIR = PROJECT_ROOT / "data" / "raw"

print(f"Project root: {PROJECT_ROOT}")
print(f"Raw data directory: {DATA_RAW_DIR}")

# 2) Collect documents by category (policies, esg, incidents)
documents_by_category = {}

for category_dir in sorted(DATA_RAW_DIR.iterdir()):
    if category_dir.is_dir():
        txt_files = sorted(category_dir.glob("*.txt"))
        documents_by_category[category_dir.name] = txt_files

# 3) Print a short summary of what we found
print("\nDiscovered raw documents:")
for category, paths in documents_by_category.items():
    print(f"- {category}: {len(paths)} files")
    for path in paths:
        print(f"  • {path.name}")

# 4) Optional: a flat list of all documents (for later steps if needed)
all_documents = [p for paths in documents_by_category.values() for p in paths]
print(f"\nTotal number of documents: {len(all_documents)}")


Project root: C:\Users\misha\OneDrive - University of Bristol\Job Apps\Concirrus\genai-insurance-risk-extraction
Raw data directory: C:\Users\misha\OneDrive - University of Bristol\Job Apps\Concirrus\genai-insurance-risk-extraction\data\raw

Discovered raw documents:
- esg: 3 files
  • esg_corporate_sustainability.txt
  • esg_energy_transition.txt
  • esg_supply_chain_governance.txt
- incident: 3 files
  • incident_marine_grounding.txt
  • incident_motor_fleet_collision.txt
  • incident_property_fire.txt
- policy: 8 files
  • auto_insurance_policy_synthetic.txt
  • businessowners_insurance_synthetic_01.txt
  • cyber_insurance_policy_synthetic.txt
  • group_life_policy_practice_unstructured_v1.txt
  • homeowners_declarations_synthetic.txt
  • homeowners_policy_ho3_synthetic.txt
  • travel_insurance_policy_synthetic_allianz.txt
  • travel_insurance_policy_synthetic_CHI.txt

Total number of documents: 14


## Step 2: Load document contents and create a simple overview

Now that we know where the files are and how many we have in each category, the next step is to actually **load the text contents** into memory and build a simple summary table.

In this step we will:

- Read each `.txt` file into a Python string  
- Capture basic metadata for each document  
  - category (policies, esg, incidents)  
  - filename  
  - full filesystem path  
  - character count  
  - word count  
  - a short text preview  
- Store everything in a pandas `DataFrame` so we can inspect the documents in a structured way


This overview will help us:

- Decide sensible chunk sizes later  
- Check that the documents have realistic lengths  
- Quickly spot any weird or empty files before we start chunking and extraction


In [2]:
import pandas as pd

# 1) Load contents of each document into memory and collect basic stats
records = []

for category, paths in documents_by_category.items():
    for path in paths:
        # Read the full text of the file
        text = path.read_text(encoding="utf-8")

        # Build a short, single line preview for quick inspection
        preview = text[:400].replace("\n", " ").strip()

        # Append a record (one row) for this document
        records.append(
            {
                "category": category,
                "filename": path.name,
                "path": path,
                "n_chars": len(text),
                "n_words": len(text.split()),
                "preview": preview,
            }
        )

# 2) Create a DataFrame with one row per document
docs_df = pd.DataFrame(records)

# 3) Show a compact summary
print("Document overview:")
display(
    docs_df[["category", "filename", "n_words", "n_chars", "preview"]]
)


Document overview:


,category,filename,n_words,n_chars,preview
0,esg,esg_corporate_sustainability.txt,714,4958,Synthetic ESG Report – Corporate Sustainabilit...
1,esg,esg_energy_transition.txt,847,5874,Synthetic ESG Report – Energy Transition and E...
2,esg,esg_supply_chain_governance.txt,823,5755,Synthetic ESG Report – Supply Chain and Climat...
3,incident,incident_marine_grounding.txt,778,4941,Synthetic Marine Incident Report – Engine Fail...
4,incident,incident_motor_fleet_collision.txt,788,5051,Synthetic Incident Report – Motor Fleet Collis...
5,incident,incident_property_fire.txt,789,5086,Synthetic Incident Report – Commercial Propert...
6,policy,auto_insurance_policy_synthetic.txt,1025,6981,Synthetic Auto Insurance Practice Document (f...
7,policy,businessowners_insurance_synthetic_01.txt,1896,12083,Synthetic Businessowners Insurance Practice Do...
8,policy,cyber_insurance_policy_synthetic.txt,1089,7425,Synthetic Cyber Insurance Practice Document (...
9,policy,group_life_policy_practice_unstructured_v1.txt,1376,8881,Synthetic Group Life Insurance Practice Docume...


## Step 3: Apply a consistent text chunking strategy

Now that the helper function has been implemented in `src/chunking.py`, we can import it and use it to split each document into manageable chunks for LLM processing.

In this project we use a word based chunking strategy with the following design choices:

- Split text into chunks of approximately 250 words.
- Use an overlap of 50 words so that important details near chunk boundaries are preserved.
- Keep the parameters flexible so that chunk sizes can be adjusted later without changing the pipeline.

In this step we will:

1. Import the `chunk_text` helper function from `src/chunking.py`.
2. Apply it to a single example document.
3. Inspect the number of chunks and preview the first few to confirm that the behaviour is sensible.


In [3]:
import sys
from pathlib import Path

# Ensure src/ is on the Python path
PROJECT_ROOT = Path("..").resolve()
sys.path.append(str(PROJECT_ROOT))

# Import the helper function
from src.chunking import chunk_text

# Test on one example document
example_row = docs_df.iloc[0]
example_text = example_row["path"].read_text(encoding="utf-8")

example_chunks = chunk_text(example_text, max_words=250, overlap=50)

print(f"Example document: {example_row['filename']}")
print(f"Total words in document: {example_row['n_words']}")
print(f"Number of chunks created: {len(example_chunks)}\n")

# Preview the first two chunks
for i, chunk in enumerate(example_chunks[:2]):
    print(f"--- Chunk {i} (first 40 words) ---")
    print(" ".join(chunk.split()[:40]))
    print()


Example document: esg_corporate_sustainability.txt
Total words in document: 714
Number of chunks created: 4

--- Chunk 0 (first 40 words) ---
Synthetic ESG Report – Corporate Sustainability Narrative (fully synthetic paraphrased text created for training and GenAI extraction testing; not based on any copyrighted ESG document) (inspired by: corporate ESG and sustainability disclosures from global manufacturing, logistics, and energy companies) The

--- Chunk 1 (first 40 words) ---
the footnotes said the reductions were influenced by lower production volumes rather than actual efficiency improvements. No single team seemed responsible for consolidating the data, which led to confusion over which version was the most accurate. The company recycled some



## Step 4: Chunk all documents and build a chunk level table

In the previous step we confirmed that the `chunk_text` helper function produces sensible chunks for a single document. The next step is to apply this function to every document in the corpus and create a structured table of chunks.

The goal of this step is to move from a **document level view** (`docs_df`) to a **chunk level view** that is suitable for LLM extraction.

In this step we will:

1. Loop over all rows in `docs_df` and apply `chunk_text` to each document.
2. For each chunk, record the following metadata:
   - `category` (policies, esg, incidents)
   - `filename`
   - `doc_index` (index of the document in `docs_df`)
   - `chunk_index` (position of the chunk within that document)
   - `chunk_text`
   - `n_words_chunk` (word count in the chunk)
3. Store all chunk records in a new pandas DataFrame called `chunks_df`.

The `chunks_df` table will have one row per chunk and will serve as the main input for the LLM extraction step. This makes it easy to track where each chunk came from and to aggregate results back to the document level later in the workflow.


In [4]:
# Step 4: Chunk all documents and build a chunk-level DataFrame

chunk_records = []

for doc_index, row in docs_df.iterrows():
    # Read full text for this document
    text = row["path"].read_text(encoding="utf-8")

    # Generate chunks using the helper
    chunks = chunk_text(text, max_words=250, overlap=50)

    # Create one record per chunk
    for chunk_index, chunk in enumerate(chunks):
        chunk_records.append(
            {
                "doc_index": doc_index,
                "category": row["category"],
                "filename": row["filename"],
                "chunk_index": chunk_index,
                "chunk_text": chunk,
                "n_words_chunk": len(chunk.split()),
            }
        )

# Build the chunk-level DataFrame
chunks_df = pd.DataFrame(chunk_records)

print("Chunk level overview:")
print(f"- Number of documents: {len(docs_df)}")
print(f"- Total number of chunks: {len(chunks_df)}")
print(f"- Average chunks per document: {len(chunks_df) / len(docs_df):.2f}\n")

display(
    chunks_df[["doc_index", "category", "filename", "chunk_index", "n_words_chunk"]]
    .head(10)
)


Chunk level overview:
- Number of documents: 14
- Total number of chunks: 76
- Average chunks per document: 5.43



,doc_index,category,filename,chunk_index,n_words_chunk
0,0,esg,esg_corporate_sustainability.txt,0,250
1,0,esg,esg_corporate_sustainability.txt,1,250
2,0,esg,esg_corporate_sustainability.txt,2,250
3,0,esg,esg_corporate_sustainability.txt,3,114
4,1,esg,esg_energy_transition.txt,0,250
5,1,esg,esg_energy_transition.txt,1,250
6,1,esg,esg_energy_transition.txt,2,250
7,1,esg,esg_energy_transition.txt,3,247
8,1,esg,esg_energy_transition.txt,4,47
9,2,esg,esg_supply_chain_governance.txt,0,250


## Step 5: Define the JSON schema and controlled vocabularies

Before calling the LLM, we need a clear and consistent definition of the structured output we expect from each chunk. This ensures that:

- The model always returns the same fields.
- Values are constrained where appropriate.
- Validation in later steps is straightforward.

In this project, each extraction call is expected to return the following JSON structure:

```json
{
  "entity_name": "",
  "region": "",
  "sector": "",
  "risk_type": "",
  "time_horizon": "",
  "key_risk_factors": [],
  "risk_summary": ""
}


In [5]:
import sys
from pathlib import Path

# Ensure src/ is on the Python path
PROJECT_ROOT = Path("..").resolve()
sys.path.append(str(PROJECT_ROOT))

# Import schema and vocabularies
from src.validation import (
    FIELD_SCHEMA,
    ALLOWED_RISK_TYPES,
    ALLOWED_REGIONS,
    ALLOWED_TIME_HORIZONS,
)

print("JSON Field Schema:")
for field, dtype in FIELD_SCHEMA.items():
    print(f"- {field}: {dtype}")

print("\nAllowed Risk Types:")
print(ALLOWED_RISK_TYPES)

print("\nAllowed Regions:")
print(ALLOWED_REGIONS)

print("\nAllowed Time Horizons:")
print(ALLOWED_TIME_HORIZONS)

JSON Field Schema:
- entity_name: string
- region: string
- sector: string
- risk_type: string
- time_horizon: string
- key_risk_factors: list
- risk_summary: string

Allowed Risk Types:
['property', 'marine', 'motor', 'cyber', 'liability', 'health', 'travel', 'esg', 'operational', 'other']

Allowed Regions:
['global', 'europe', 'north_america', 'asia_pacific', 'latin_america', 'middle_east_africa']

Allowed Time Horizons:
['short_term', 'medium_term', 'long_term', 'multi_horizon', 'not_specified']


## Step 6: Load and test the LLM extraction prompt template

With the JSON schema and controlled vocabularies defined, the next step is to use a clear, repeatable prompt template for the LLM. The goal is to ensure that every call to LLaMA 3 (via the Groq API) returns a strict JSON object that matches the agreed schema and uses only the allowed values where specified.

To keep the notebook focused on workflow rather than long prompt strings, the prompt template is defined inside `src/prompt_template.py`. That helper module:

- Stores the base prompt text, including the JSON schema and constraints.
- Inserts the controlled vocabularies for `risk_type`, `region`, and `time_horizon`.
- Provides a `build_prompt()` function that takes a single chunk of text and returns a fully formatted prompt.
- Enforces that the model should return valid JSON only, with no extra commentary or invented information.

In this step we will:

1. Import the `build_prompt` helper from `src/prompt_template.py`.
2. Apply it to one example chunk from `chunks_df`.
3. Inspect the first part of the resulting prompt to confirm that:
   - the schema is included,
   - the allowed values are correctly inserted,
   - the chunk content appears in the intended place.

This prompt template will act as the standard interface between the chunk level inputs and the LLM extraction logic that we implement in the next step.


In [6]:
from src.prompt_template import build_prompt

# Test prompt building on the first chunk
test_chunk = chunks_df.loc[0, "chunk_text"]
test_prompt = build_prompt(test_chunk)

print(test_prompt[:800])


You are an AI assistant that extracts structured risk information from insurance related text.

Your task:
- Read the provided document chunk carefully.
- Extract the requested fields based only on information in this chunk.
- If a field is not mentioned or cannot be inferred with high confidence, leave it as an empty string "" or an empty list [].

Output format:
Return a single JSON object with exactly these fields:
- entity_name (string)
- region (string, one of: global, europe, north_america, asia_pacific, latin_america, middle_east_africa)
- sector (string)
- risk_type (string, one of: property, marine, motor, cyber, liability, health, travel, esg, operational, other)
- time_horizon (string, one of: short_term, medium_term, long_term, multi_horizon, not_specified)
- key_risk_factors (


## Step 7: Wire up the LLM extraction helper and test on a single chunk

With the prompt template and JSON schema in place, the next step is to connect the notebook to the LLM extraction helper implemented in `src/extraction.py`. The aim is to send a single chunk of text to LLaMA 3 (via the Groq API), receive a structured JSON response, and validate it against the expected schema.

The extraction helper is responsible for:

1. Loading the Groq API key from the `.env` file using `python-dotenv`.
2. Creating a Groq client that will send requests to the LLaMA 3 model.
3. Building the final prompt for a given chunk using `build_prompt` from `src/prompt_template.py`.
4. Calling the Groq chat completion API with:
   - a fixed system message that describes the model’s role as a JSON extraction assistant
   - a user message containing the formatted prompt and chunk text
5. Parsing the model output as JSON.
6. Validating the parsed JSON using `validate_extracted_json` from `src/validation.py`.
7. Returning a cleaned Python dictionary that matches the agreed schema, or `None` if extraction fails after the allowed number of retries.

In this step we will:

1. Import the `call_llm_on_chunk` helper from `src/extraction.py`.
2. Select a single example chunk from `chunks_df`.
3. Run the helper once on that chunk.
4. Inspect the returned dictionary to confirm that:
   - all expected fields are present,
   - controlled fields (such as `risk_type` and `region`) use allowed values, and
   - the summary fields (`key_risk_factors`, `risk_summary`) look reasonable for the input text.

This step serves as a sanity check before running the extraction over all chunks in a later step. It confirms that the end to end flow from chunk text, through prompt building, API call, JSON parsing, and validation is working correctly on a small example.


In [7]:
from src.extraction import call_llm_on_chunk

# Select a single example chunk to test the full extraction pipeline
example_row = chunks_df.iloc[0]
example_chunk_text = example_row["chunk_text"]

print("Document index:", example_row["doc_index"])
print("Category:", example_row["category"])
print("Filename:", example_row["filename"])
print("\nFirst 300 characters of the chunk:\n")
print(example_chunk_text[:300])
print("\n" + "=" * 80 + "\n")

# Call the LLM extraction helper on this single chunk
extracted_dict = call_llm_on_chunk(example_chunk_text, max_retries=1)

print("Extraction result (Python dict):\n")
print(extracted_dict)

# If the result is not None, show the keys to confirm the schema
if extracted_dict is not None:
    print("\nKeys in the extracted dictionary:")
    print(list(extracted_dict.keys()))
else:
    print("\nExtraction failed or returned None.")


Document index: 0
Category: esg
Filename: esg_corporate_sustainability.txt

First 300 characters of the chunk:

Synthetic ESG Report – Corporate Sustainability Narrative (fully synthetic paraphrased text created for training and GenAI extraction testing; not based on any copyrighted ESG document) (inspired by: corporate ESG and sustainability disclosures from global manufacturing, logistics, and energy compan


Extraction result (Python dict):

{'entity_name': 'global manufacturing, logistics, and energy companies', 'region': 'global', 'sector': 'manufacturing, logistics, and energy', 'risk_type': 'esg', 'time_horizon': 'not_specified', 'key_risk_factors': ['inconsistent ESG strategy', 'different interpretations of sustainability across regions', 'inconsistent documentation', 'difficulty in comparing environmental impacts', 'reliability issues with contractor-operated equipment', 'confusion over data accuracy'], 'risk_summary': "The company's ESG strategy is still developing and not co

## Step 8: Test extraction on a small sample of chunks

Before running the LLM across the entire corpus, it is important to test the full extraction pipeline on a small, diverse sample of chunks. This acts as a smoke test for both the prompt design and the `call_llm_on_chunk` helper.

In this step we will:

1. Select a small subset of chunks from `chunks_df`, covering different document categories (policies, esg, incidents).
2. Run `call_llm_on_chunk` on each selected chunk.
3. Inspect the returned dictionaries to confirm that:
   - All expected fields are present.
   - `risk_type`, `region`, and `time_horizon` values come from the allowed vocabularies.
   - `key_risk_factors` and `risk_summary` are reasonable given the input text.
4. Log any failures or strange outputs so that we can adjust the prompt or validation logic before scaling up.

This step gives confidence that the end to end extraction pipeline behaves sensibly across different document types, before we move on to the full Map step over all chunks.


In [8]:
# Step 8: Test extraction on a small sample of chunks

from src.extraction import call_llm_on_chunk

# 1) Pick a small, diverse sample of chunks
#    Here we take the first 2 chunks from each category, if available.
sample_chunks = (
    chunks_df
    .groupby("category", group_keys=False)
    .head(2)
    .reset_index(drop=True)
)

print("Sample size for manual inspection:", len(sample_chunks))
display(
    sample_chunks[["doc_index", "category", "filename", "chunk_index", "n_words_chunk"]]
)

# 2) Run the LLM extraction helper on each sampled chunk
results = []

for idx, row in sample_chunks.iterrows():
    print("\n" + "=" * 80)
    print(f"Sample {idx+1} of {len(sample_chunks)}")
    print(f"Category      : {row['category']}")
    print(f"Filename      : {row['filename']}")
    print(f"Document index: {row['doc_index']}")
    print(f"Chunk index   : {row['chunk_index']}")
    print(f"Chunk length  : {row['n_words_chunk']} words")
    print("-" * 80)
    print("Chunk preview (first 250 characters):\n")
    print(row["chunk_text"][:250])
    print("\nCalling LLM extraction helper...\n")

    chunk_text = row["chunk_text"]

    extracted = call_llm_on_chunk(chunk_text, max_retries=2)
    results.append(extracted)

    print("Extraction result (summary):")
    if extracted is None:
        print("  Extraction failed or returned None.")
    else:
        # Safely get key fields with defaults
        entity_name = extracted.get("entity_name", "")
        region = extracted.get("region", "")
        risk_type = extracted.get("risk_type", "")
        time_horizon = extracted.get("time_horizon", "")
        risk_summary = extracted.get("risk_summary", "")

        print(f"  entity_name  : {entity_name}")
        print(f"  region       : {region}")
        print(f"  risk_type    : {risk_type}")
        print(f"  time_horizon : {time_horizon}")
        print("  risk_summary :")
        print(" ", risk_summary[:300])

# Optional: keep results together for later inspection if needed
sample_extractions = results


Sample size for manual inspection: 6


,doc_index,category,filename,chunk_index,n_words_chunk
0,0,esg,esg_corporate_sustainability.txt,0,250
1,0,esg,esg_corporate_sustainability.txt,1,250
2,3,incident,incident_marine_grounding.txt,0,250
3,3,incident,incident_marine_grounding.txt,1,250
4,6,policy,auto_insurance_policy_synthetic.txt,0,250
5,6,policy,auto_insurance_policy_synthetic.txt,1,250



Sample 1 of 6
Category      : esg
Filename      : esg_corporate_sustainability.txt
Document index: 0
Chunk index   : 0
Chunk length  : 250 words
--------------------------------------------------------------------------------
Chunk preview (first 250 characters):

Synthetic ESG Report – Corporate Sustainability Narrative (fully synthetic paraphrased text created for training and GenAI extraction testing; not based on any copyrighted ESG document) (inspired by: corporate ESG and sustainability disclosures from 

Calling LLM extraction helper...

Extraction result (summary):
  entity_name  : global manufacturing, logistics, and energy companies
  region       : global
  risk_type    : esg
  time_horizon : not_specified
  risk_summary :
  The company's ESG strategy is still developing and not consistently implemented across business units, creating an inconsistent picture when trying to present a single global ESG narrative. Different regions operate with their own interpretations of wha

## Step 9: Map phase across all chunks

Now that the extraction helper has been tested on individual chunks and on a small, diverse sample, we can scale the process to the full corpus. The goal of this step is to apply the LLM extraction function to every chunk in `chunks_df`.

Each row in `chunks_df` represents a small slice of a longer document. In the Map phase we send each slice through the LLM and collect the structured JSON outputs. This produces a new DataFrame called `chunk_outputs_df` that contains, for every chunk:

- The document index and filename  
- The document category (`policies`, `esg`, or `incidents`)  
- The chunk index within that document  
- The extracted JSON dictionary returned by the LLM  

In this step we will:

1. Loop over all rows in `chunks_df`.  
2. For each row, pass the `chunk_text` to `call_llm_on_chunk`.  
3. Store the returned Python dictionary together with the document and chunk identifiers.  
4. Record any failures as `None` so that they can be handled safely later.  
5. Convert the collected results into a new DataFrame called `chunk_outputs_df`.  

Some chunks will not contain clear risk information or the LLM may occasionally fail. By keeping these as `None`, the pipeline remains robust and transparent. Once this Map phase is complete, we will have one row per chunk with its corresponding structured output. 

In the next step, we will move to the Reduce phase, where chunk level outputs are merged into a single document level record for each original file.


In [10]:
## Step 9: Map phase over all chunks (using tqdm progress bar)

from src.extraction import call_llm_on_chunk
from tqdm import tqdm
import time

map_records = []

total_chunks = len(chunks_df)
print(f"Starting Map phase over all chunks (total: {total_chunks})")

start_time = time.time()

# Wrap the iterator with tqdm for a progress bar
for idx, row in tqdm(chunks_df.iterrows(), total=total_chunks):
    chunk_text = row["chunk_text"]

    extracted = call_llm_on_chunk(chunk_text, max_retries=2)

    map_records.append(
        {
            "doc_index": row["doc_index"],
            "category": row["category"],
            "filename": row["filename"],
            "chunk_index": row["chunk_index"],
            "extracted_json": extracted,
        }
    )

end_time = time.time()
print(f"Map phase complete in {end_time - start_time:.2f} seconds")

# Build the chunk output DataFrame
chunk_outputs_df = pd.DataFrame(map_records)

print("\nChunk extraction output overview:")
display(
    chunk_outputs_df[
        ["doc_index", "category", "filename", "chunk_index", "extracted_json"]
    ].head(10)
)

# Failure count
n_failed = chunk_outputs_df["extracted_json"].isna().sum()
print(f"\nTotal chunks: {total_chunks}")
print(f"Number of failed extractions: {n_failed}")


Starting Map phase over all chunks (total: 76)


100%|██████████████████████████████████████████████████████████████████████████████████| 76/76 [08:27<00:00,  6.68s/it]

Map phase complete in 507.74 seconds

Chunk extraction output overview:


,doc_index,category,filename,chunk_index,extracted_json
0,0,esg,esg_corporate_sustainability.txt,0,"{'entity_name': 'global manufacturing, logisti..."
1,0,esg,esg_corporate_sustainability.txt,1,"{'entity_name': '', 'region': '', 'sector': 'i..."
2,0,esg,esg_corporate_sustainability.txt,2,"{'entity_name': '', 'region': '', 'sector': 'i..."
3,0,esg,esg_corporate_sustainability.txt,3,"{'entity_name': '', 'region': 'global', 'secto..."
4,1,esg,esg_energy_transition.txt,0,"{'entity_name': 'The organisation', 'region': ..."
5,1,esg,esg_energy_transition.txt,1,"{'entity_name': '', 'region': 'global', 'secto..."
6,1,esg,esg_energy_transition.txt,2,"{'entity_name': '', 'region': '', 'sector': 'e..."
7,1,esg,esg_energy_transition.txt,3,"{'entity_name': '', 'region': 'global', 'secto..."
8,1,esg,esg_energy_transition.txt,4,"{'entity_name': '', 'region': '', 'sector': 'E..."
9,2,esg,esg_supply_chain_governance.txt,0,{'entity_name': 'Synthetic ESG Report – Supply...



Total chunks: 76
Number of failed extractions: 0


## Step 10: Reduce phase from chunk level to document level

After completing the Map phase, we now have `chunk_outputs_df`, which contains one row per chunk and a column called `extracted_json` that stores the structured output from the LLM. The goal of this step is to merge all chunk level outputs that belong to the same document into a single consolidated record.

This is the Reduce phase in the MapReduce pattern. Instead of treating each chunk independently, we aggregate the information from all chunks of a document to produce one clean document level summary.

In this step we will:

1. Group the chunk outputs by `doc_index` so that all chunks from the same document are processed together.  
2. For each document, merge the `extracted_json` dictionaries using simple rules:  
   - For fields such as `entity_name`, `region`, `sector`, `risk_type`, and `time_horizon`, take the first non empty value or the most common value across chunks.  
   - For `key_risk_factors`, combine lists from all chunks and remove duplicates.  
   - For `risk_summary`, join short summaries from multiple chunks into one concise paragraph.  
3. Construct a new DataFrame called `docs_extracted_df` that contains one row per original document, along with its merged fields.  
4. Inspect a few rows to confirm that the merging strategy gives sensible results.  
5. Save the merged document level outputs into `data/processed` for use in Notebook 02 (EDA) and Notebook 03 (feature engineering and classification).

By the end of this step we will have transformed the noisy chunk level data into clean document level signals. This provides a reliable foundation for downstream analysis, mirroring how real insurtech pipelines summarise multiple paragraph level extractions into a single risk oriented view of each document.


In [11]:
## Step 10: Reduce phase from chunk level to document level

from src.reducer import reduce_extractions_for_document

def reduce_document_group(group: pd.DataFrame) -> pd.Series:
    """
    Reduce all chunk level extractions for a single document into one row.
    This wraps the reduce_extractions_for_document helper and adds metadata.
    """
    # Collect all valid dicts from the extracted_json column
    chunk_dicts = [
        d for d in group["extracted_json"]
        if isinstance(d, dict)
    ]

    # Build simple metadata for this document
    doc_meta = {
        "doc_index": int(group["doc_index"].iloc[0]),
        "category": group["category"].iloc[0],
        "filename": group["filename"].iloc[0],
        "n_chunks": len(group),
        "n_chunks_with_data": len(chunk_dicts),
    }

    # Call the reducer helper to merge all chunk level dicts
    doc_level = reduce_extractions_for_document(
        chunk_extractions=chunk_dicts,
        doc_metadata=doc_meta,
    )

    # Convert dict to Series so that groupby.apply can stack them
    return pd.Series(doc_level)


# Apply the reducer to every document in chunk_outputs_df
docs_extracted_df = (
    chunk_outputs_df
    .groupby("doc_index", group_keys=False)
    .apply(reduce_document_group)
    .reset_index(drop=True)
)

print("Document level extraction overview:")
display(
    docs_extracted_df[
        [
            "doc_index",
            "category",
            "filename",
            "n_chunks",
            "n_chunks_with_data",
            "entity_name",
            "region",
            "sector",
            "risk_type",
            "time_horizon",
            "key_risk_factors",
        ]
    ]
)

print("\nExample merged risk_summary for the first document:\n")
print(docs_extracted_df.loc[0, "risk_summary"])


# Optional: save document level outputs to data/processed for later notebooks
DATA_PROCESSED_DIR = PROJECT_ROOT / "data" / "processed"
DATA_PROCESSED_DIR.mkdir(parents=True, exist_ok=True)

output_csv = DATA_PROCESSED_DIR / "docs_extracted.csv"
output_json = DATA_PROCESSED_DIR / "docs_extracted.json"

docs_extracted_df.to_csv(output_csv, index=False)
docs_extracted_df.to_json(output_json, orient="records", indent=2)

print(f"\nSaved document level CSV to: {output_csv}")
print(f"Saved document level JSON to: {output_json}")


Document level extraction overview:


C:\Users\misha\AppData\Local\Temp\ipykernel_5628\2370556573.py:39: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(reduce_document_group)


,doc_index,category,filename,n_chunks,n_chunks_with_data,entity_name,region,sector,risk_type,time_horizon,key_risk_factors
0,0,esg,esg_corporate_sustainability.txt,4,4,"global manufacturing, logistics, and energy co...",global,industrial,esg,not_specified,"[inconsistent ESG strategy, different interpre..."
1,1,esg,esg_energy_transition.txt,5,5,The organisation,global,"energy, logistics, heavy industry, transport",esg,medium_term,[inconsistency in language used in internal do...
2,2,esg,esg_supply_chain_governance.txt,5,5,Synthetic ESG Report – Supply Chain and Climat...,global,energy,esg,not_specified,[inconsistent expectations in environmental cr...
3,3,incident,incident_marine_grounding.txt,4,4,,global,marine,property,not_specified,"[engine failure, grounding, irregular vibratio..."
4,4,incident,incident_motor_fleet_collision.txt,4,4,Motor Fleet Collision Event,global,transportation,motor,not_specified,"[driver error, vehicle maintenance, sensor cal..."
5,5,incident,incident_property_fire.txt,4,4,,north_america,manufacturing,property,not_specified,"[faulty fire alarm panel, unusual smell, contr..."
6,6,policy,auto_insurance_policy_synthetic.txt,6,6,Synthetic Auto Insurance,north_america,insurance,liability,short_term,"[bodily injury, property damage, liability, fa..."
7,7,policy,businessowners_insurance_synthetic_01.txt,10,10,,north_america,insurance,liability,not_specified,"[exclusions, expected losses, expected or inte..."
8,8,policy,cyber_insurance_policy_synthetic.txt,6,6,Cyber Insurance,global,insurance,cyber,not_specified,"[malicious attacks, accidental system failures..."
9,9,policy,group_life_policy_practice_unstructured_v1.txt,7,7,Synthetic Group Life Insurance,global,insurance,liability,not_specified,"[discrepancies in reported earnings, late enro..."



Example merged risk_summary for the first document:

The company's ESG strategy is still developing and not consistently implemented across business units, leading to an inconsistent picture when presenting a single global ESG narrative. Different regions have their own interpretations of sustainability, and the level of documentation varies. The company has attempted to quantify environmental impacts, but the numbers are difficult to compare due to changing boundaries and assumptions. The company's operational risk is high due to inconsistent data collection, vague details on recycling, and estimated waste reduction. The lack of formal policy on waste reduction and inconsistent reporting on social issues also contribute to the risk. Additionally, incomplete supporting documents for safety metrics and year to date estimates for safety metric ...

Saved document level CSV to: C:\Users\misha\OneDrive - University of Bristol\Job Apps\Concirrus\genai-insurance-risk-extraction\data\process

## Final Summary and Next Steps

Notebook 01 is now complete. In this notebook we built an end to end GenAI extraction pipeline that prepares unstructured insurance style documents for downstream analysis. The workflow followed a clear MapReduce pattern:

1. Load and inspect all raw documents.  
2. Apply a consistent sliding window chunking strategy.  
3. Define a strict JSON schema with controlled vocabularies.  
4. Build a stable prompt template for structured extraction.  
5. Run the Map phase by calling the LLM on every chunk.  
6. Validate outputs and collect chunk level results.  
7. Run the Reduce phase to combine chunk level dicts into one document level record.  
8. Save the final merged dataset to `data/processed` for EDA and modelling.

This gives us a clean table with one row per document and consistent fields such as entity name, region, sector, risk type, and extracted risk factors. These outputs form the foundation for Notebook 02, where we will explore the distributions of the extracted fields, inspect missingness patterns, measure text lengths, and begin building domain informed features.

The next notebook will focus on EDA and normalisation, and will help us understand how the extracted signals vary across policy documents, ESG reports, and incident summaries. This mirrors the exploratory work an insurtech team would perform before building any risk scoring or classification models.
